###  Mini Project - Hand Writting Recognition

**Data Prep, Training and Evaluation**

In [6]:
import cv2
import numpy as np


image = cv2.imread(r'C:\Users\princ\Documents\Mini openCV projects\image_for_computer_vision\digits.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
small = cv2.pyrDown(image)
cv2.imshow('Dataset Image', small)
cv2.waitKey(0)
cv2.destroyAllWindows()


# code that splits the image to 5000 cells, each of size 20 x 20 size 
# this gives us a 4D array: 50 x 100 x 20 x20 i.e 50 rows and 100 cols where each row col is 20 x 20 grid  containing the number pixel
# done using the horizontal split (hsplit) and vertical split(vsplit) of numpy
cells = [np.hsplit(row, 100) for row in np.vsplit(gray, 50)]

# converts the cell into a numpy array of shape(50, 100, 20, 20)
x = np.array(cells)
print('The shape of our cell array: ' + str(x.shape))


# split data set into two segments
# one for training the model and the other for testing the data set.
# e.g test dataset has shape (1500, 400) where 1500 now reps the instances
# and 400 reps the flattened 20 x 20 pixel repping the instance or number
train = x[:,:70].reshape(-1, 400).astype(np.float32)
test = x[:,70:100].reshape(-1, 400).astype(np.float32)

# creates labels for the train and test data set
k = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
train_labels = np.repeat(k, 350)[:, np.newaxis]
test_labels = np.repeat(k, 150)[:, np.newaxis]

# initialize the KNN object and, train it with the train data set and test it with the test data for k= 3
knn = cv2.ml.KNearest_create()
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
ret, result, neighbours, distance = knn.findNearest(test, k=3)

# checks the accuracy of classification
# the macthes var will be equal to the number of times our result was equal to our test_labels.
matches = result == test_labels

# counts how many matches that were not found.
correct = np.count_nonzero(matches)
accuracy = correct * (100.0 / result.size)
print('Accuracy is = %.2f' % accuracy + '%')

The shape of our cell array: (50, 100, 20, 20)
Accuracy is = 93.47%


**Defines Helper functions used to prepare the input image**

In [9]:
import cv2
import numpy as np


def x_cord_contour(contour):
    """
    takes a countour and returns the centroid x-cordinates as output
    also it does this by the default left to right sorting.
    """
    if cv2.contourArea(contour) > 10:
        M = cv2.moments(contour)
        return (int(M['m10']/ M['m00']))
    
    

def makeSquare(not_square):
    """
    takes the input image and makes the dimensions sqaure
    and also adds the black pixels padding where needed.
    """
    BLACK = [0, 0, 0]
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    print('Height= ', height, 'width= ', width)
    if height == width:
        square = not_square
        return square
    else:
        doublesize = cv2.resize(not_square, (2*width, 2*height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        print('New Height', height, 'New width = ', width)
        
        if height > width:
            pad = int((height - width)/2)
            print('Padding= ', pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,0,0,pad,\
                                                   pad, cv2.BORDER_CONSTANT, value=BLACK)
        else:
            pad = int((width - height)/2)
            print ('Padding= ', pad)
            doublesize_square = cv2.copyMakeBorder(double_size, pad, pad, 0, 0,\
                                                  cv2.BORDER_CONSTANT, value=BLACK)
    doublesize_square_dim = doublesize_square.shape
    print('Sq Height = ', doublesize_square_dim[0], 'sq width= ', doublesize_square_dim[1]) 
    return doublesize_square



def resize_to_pixel(dimensions, image):
    """
    resizes our image to the specified dimensions 
    
    Args:
        dimension : Dimension to resize the image to
        image: An image having it's height equal to it's width will be operated on
    """
    buffer_pix = 4
    dimensions = dimensions - buffer_pix
    squared = image
    r = float(dimensions) / squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    width_r = img_dim2[1]
    BLACK = [0,0,0]
    
    if height_r > width_r:
        resized = cv2.copyMakeBorder(resized,0,0,0,1,cv2.BORDER_CONSTANT, value=BLACK)
    if height_r < width_r:
        resized = cv2.copyMakeBorder(resized,1,0,0,0,cv2.BORDER_CONSTANT, value = BLACK)
        
    p=2
    ResizedImg = cv2.copyMakeBorder(resized, p,p,p,p,cv2.BORDER_CONSTANT, value = BLACK)
    img_dim = ResizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    print('Padded Height= ', height, 'Width= ', width, "\n") 
    return ResizedImg
        


**Loads input image, processes it and classifies the digits found on the input image**

In [10]:
import cv2
import numpy as np

image = cv2.imread(r'C:\Users\princ\Documents\Mini openCV projects\image_for_computer_vision\Numbers1.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('image', image)
cv2.imshow('Grey_img', gray)
cv2.waitKey(0)

# blurs the image and finds the edges using the canny algo
blurred = cv2.GaussianBlur(gray, (5,5),0)
cv2.imshow('Blurred', blurred)
cv2.waitKey()

edged = cv2.Canny(blurred, 30, 150)
cv2.imshow('Edged', edged)
cv2.waitKey()


# finds contours
contours, _= cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# code that sorts our contours from left to right.
# contours = sorted(contours, key=x_cord_contour, reverse=False)

# code that creates an array to store all the numbers
full_number = []

# code that iterates over thhe countours found
for c in contours:
    # computes the bounding rect of each contour rep a number
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.drawContours(image, contours, -1, (0, 255, 100), 3)
    cv2.imshow('Contours', image)
    cv2.waitKey()
    
    
    if w >= 5 and h >= 25:
        roi = blurred[y:y+h, x:x+w]
        ret, roi = cv2.threshold(roi, 127, 255, cv2.THRESH_BINARY_INV)
        squared = makeSquare(roi)
        final = resize_to_pixel(20, squared)
        
        cv2.imshow('Final', final)
        cv2.waitKey()
        
        final_array = final.reshape((1, 400))  # flattens the image from 20 x 20 to 1 x 400
        final_array = final_array.astype(np.float32)
        ret, result, neighbours, dist = knn.findNearest(final_array, k=1)
        number = str(int(float(result[0])))
        full_number.append(number)
        
        # code tht draws a rect around the digit, this shows what the digit was classified as
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0,0), 2)
        cv2.putText(image, number, (x, y +150), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 2)
        cv2.imshow('Image', image)
        cv2.waitKey
        
cv2.destroyAllWindows()
print('The Number is : ' + ''.join(reversed(full_number)))
        

Height=  98 width=  68
New Height 196 New width =  136
Padding=  30
Sq Height =  196 sq width=  196
Padded Height=  20 Width=  20 

Height=  96 width=  66
New Height 192 New width =  132
Padding=  30
Sq Height =  192 sq width=  192
Padded Height=  20 Width=  20 

Height=  91 width=  48
New Height 182 New width =  96
Padding=  43
Sq Height =  182 sq width=  182
Padded Height=  20 Width=  20 

Height=  82 width=  72
New Height 164 New width =  144
Padding=  10
Sq Height =  164 sq width=  164
Padded Height=  20 Width=  20 

Height=  103 width=  18
New Height 206 New width =  36
Padding=  85
Sq Height =  206 sq width=  206
Padded Height=  20 Width=  20 

The Number is : 13540
